In [2]:
using JacobMPS

In [26]:
σz = Float64[1 0; 0 -1];
OpA=σz;

L=16
#initial MPO for evolution 
NN = [zeros(Complex{Float64},1,2,2,1) for i=1:L];
 for i=1:L
        NN[i][1,1,1,1] = 1.0/sqrt(2);
        NN[i][1,2,2,1] = 1.0/sqrt(2);
    NN[i][1,1,2,1] = 0.0;
    NN[i][1,2,1,1] = 0.0;    
 end
MPOA=TMPS(copy(NN),3);
    MPOA.A[3][1,:,:,1]=OpA/sqrt(2)*2;



In [20]:
norm(MPOA)

0.9999999999999999

In [28]:
normalizeMPS!(MPOA)

3.9999999999999982

In [27]:
totnormsq(MPOA)

3.99999999999999 + 0.0im

In [3]:
using TensorOperations
function normalizeMPS!(ψ::TMPS)
    if ψ.oc==length(ψ.A)
        moveto!(ψ,length(ψ.A)-1)
    end
    oc=ψ.oc
    Ai=ψ.A[oc]
    Ai1=ψ.A[oc+1]
    (lbd,phyd,auxd,mbd)=size(Ai)
    (mbd,phyd,auxd,rbd)=size(Ai1)
    @tensor AiAi1[lb,phy1,aux1,phy2,aux2,rb]:=Ai[lb,phy1,aux1,md]*Ai1[md,phy2,aux2,rb]
    AA=reshape(AiAi1,lbd*phyd*auxd,phyd*auxd*rbd)
    #do svd find the schmidt value
    (u,d,v) = svd(AA)
    mm=mbd;
    d = d[1:mm]
    U = u[:,1:mm]
    V = v[:,1:mm]'
    #determine the norm
    totnorm=dot(d,d) 
    d=d/sqrt(totnorm) #normalize the MPS
    U = U * diagm(d) #put the OC toleft
    #put the matrices back
    ψ.A[oc]=reshape(U,lbd,phyd,auxd,mbd)
    ψ.A[oc+1]=reshape(V,mbd,phyd,auxd,rbd)
    return (totnorm)
end




normalizeMPS! (generic function with 1 method)

In [12]:
@time normsq(MPOA)

  0.000116 seconds (249 allocations: 11.609 KiB)


0.9999999999999998 + 0.0im

In [14]:
@time MPSdot(MPOA,MPOA)

  0.008108 seconds (46.64 k allocations: 2.178 MiB)


0.9999999999999799 + 0.0im